In [1]:
import sys

sys.path.append('c:/users/shubh/programming_work/virtual_envs/genai_exp_env/lib/site-packages')


In [2]:
import json
import copy
import numpy as np
import os
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
import os
import time
import pandas as pd
from fastembed import TextEmbedding
import scipy.spatial.distance as spdist
from langchain_core.retrievers import BaseRetriever
from typing import List
from langchain_openai import ChatOpenAI
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.runnables import RunnableLambda
from typing import Dict
from openai import OpenAI
import string
from collections import Counter
import re

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:

def strip_special_chars(s):
    # Using regex to remove leading and trailing non-alphanumeric characters
    return re.sub(r'^[^A-Za-z0-9]+|[^A-Za-z0-9]+$', '', s)


In [4]:
with open('openai_key.txt', 'r') as f:
    OPENAI_KEY = f.read().strip()


In [5]:
client = OpenAI(api_key = OPENAI_KEY)

In [6]:
def chat_completion(prompt, temperature = 0.1):

    global client
    return client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=temperature
    ).choices[0].message.content

In [7]:
def chat_completion_2(message_history, temperature = 0.1):

    global client
    return client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=message_history,
        temperature=temperature
    ).choices[0].message.content

In [8]:

youtube_comment_categories = [
    "Humor and Memes",
    "Appreciation and Praise",
    "Words of encouragement",
    "Words of empathy",
    "Personal Stories and Experiences",
    "Nostalgia and Memories"
]


comment_category_2_bool = dict()
for cat in youtube_comment_categories:
    comment_category_2_bool[cat] = 0

comment_category_2_bool

{'Humor and Memes': 0,
 'Appreciation and Praise': 0,
 'Words of encouragement': 0,
 'Words of empathy': 0,
 'Personal Stories and Experiences': 0,
 'Nostalgia and Memories': 0}

In [9]:
df = pd.read_csv("spotify_youtube_comments_combined.csv")

In [10]:
len(df)

80467

In [11]:
df['unique_id'] = [_ for _ in range( len(df) )]

In [12]:
df.head()

,Unnamed: 0,Artist,Url_spotify,Track,Album,Album_type,Uri,Danceability,Energy,Key,...,Url_youtube,Title,spotify_id,id,lyrics,youtube_video_id,video_id,comment,likes,unique_id
0,0,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,Feel Good Inc.,Demon Days,album,spotify:track:0d28khcov6AiegSCpG5TuT,0.818,0.705,6.0,...,https://www.youtube.com/watch?v=HyHNuVaZJ-k,Gorillaz - Feel Good Inc. (Official Video),0d28khcov6AiegSCpG5TuT,0d28khcov6AiegSCpG5TuT,"Feel good\n Sha, sha-ba-da, sha-ba-da-ca, feel...",HyHNuVaZJ-k,HyHNuVaZJ-k,I love how Gorillaz has depressing meanings to...,14760.0,0
1,1,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,Feel Good Inc.,Demon Days,album,spotify:track:0d28khcov6AiegSCpG5TuT,0.818,0.705,6.0,...,https://www.youtube.com/watch?v=HyHNuVaZJ-k,Gorillaz - Feel Good Inc. (Official Video),0d28khcov6AiegSCpG5TuT,0d28khcov6AiegSCpG5TuT,"Feel good\n Sha, sha-ba-da, sha-ba-da-ca, feel...",HyHNuVaZJ-k,HyHNuVaZJ-k,It’s really crazy to realize how popular this ...,14226.0,1
2,2,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,Feel Good Inc.,Demon Days,album,spotify:track:0d28khcov6AiegSCpG5TuT,0.818,0.705,6.0,...,https://www.youtube.com/watch?v=HyHNuVaZJ-k,Gorillaz - Feel Good Inc. (Official Video),0d28khcov6AiegSCpG5TuT,0d28khcov6AiegSCpG5TuT,"Feel good\n Sha, sha-ba-da, sha-ba-da-ca, feel...",HyHNuVaZJ-k,HyHNuVaZJ-k,Feel good inc. sounds like it should either be...,12339.0,2
3,3,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,Feel Good Inc.,Demon Days,album,spotify:track:0d28khcov6AiegSCpG5TuT,0.818,0.705,6.0,...,https://www.youtube.com/watch?v=HyHNuVaZJ-k,Gorillaz - Feel Good Inc. (Official Video),0d28khcov6AiegSCpG5TuT,0d28khcov6AiegSCpG5TuT,"Feel good\n Sha, sha-ba-da, sha-ba-da-ca, feel...",HyHNuVaZJ-k,HyHNuVaZJ-k,The art style is just so intriguing. It’s like...,10088.0,3
4,4,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,Feel Good Inc.,Demon Days,album,spotify:track:0d28khcov6AiegSCpG5TuT,0.818,0.705,6.0,...,https://www.youtube.com/watch?v=HyHNuVaZJ-k,Gorillaz - Feel Good Inc. (Official Video),0d28khcov6AiegSCpG5TuT,0d28khcov6AiegSCpG5TuT,"Feel good\n Sha, sha-ba-da, sha-ba-da-ca, feel...",HyHNuVaZJ-k,HyHNuVaZJ-k,The way Murdocs bass playing is animated is so...,9490.0,4


In [13]:

comments = list(df['comment'])
# comments

In [14]:
def check_if_categories_are_proper(llm_suggested_categories, message_history, counter, total_trials = 0):

    if total_trials==3:
        return (llm_suggested_categories, False)
    
    global youtube_comment_categories

    global required_retries_list_of_tuples 
    
    categories_list = [ strip_special_chars(category.strip(" ").strip("'").strip("\n") ) for category in llm_suggested_categories]

    list_of_remaining_categories = list(set(categories_list) - set(youtube_comment_categories))
    
    if (len(list_of_remaining_categories )>0):

        # llm_suggested_categories = chat_completion_2(prompt, temperature = 0.1).split(",")
        print(f"Requires retries - {llm_suggested_categories}")
        print(f"Requires retries list - {categories_list}")
        print(f"Requires retries remaining categories - {list_of_remaining_categories}")
        message_history.append({"role":"assistant", "content":f"{llm_suggested_categories}"})
        message_history.append({"role":"user", "content": f"Restrict the comma-separated response only to the list of relevant categories. Try again."})
        llm_suggested_categories = chat_completion_2(message_history, temperature = 0.1).strip("[").strip("]").split(",")
        llm_suggested_categories, sanity_check_passed_or_not = check_if_categories_are_proper( llm_suggested_categories, message_history, counter, total_trials = total_trials+1)
        
        required_retries_list_of_tuples.append( ( message_history, total_trials, counter ) )        
        
    else:
        return (llm_suggested_categories, True)
        
    return (llm_suggested_categories, sanity_check_passed_or_not)

In [15]:
def retry_until_you_get_correct_categories(total_trials = 0):
    pass

In [16]:
df.to_csv("C:/Users/shubh/programming_work/git_repos/NU/courses_and_academics/data_mining/proj/refined/df_indexed.csv")

In [17]:
list_of_tuples_of_skipped_indices_and_comments = list()

In [18]:
list_of_category_2_bools_per_comment = list()
required_retries_list_of_tuples = list()

In [19]:

file_path = "C:/Users/shubh/programming_work/git_repos/NU/courses_and_academics/data_mining/proj/refined/index_comment_categories_recent.csv"


In [20]:
df_index_comment_categories = pd.DataFrame()
does_df_exist = False

In [21]:
if ( os.path.exists(file_path) and file_path.endswith('.csv') ):
    print("found it")
    df_index_comment_categories = pd.read_csv(file_path, index_col=False)
    does_df_exist = True

else:
    print("it's not there yet")
    df_index_comment_categories = pd.DataFrame()

found it


In [22]:
counter = len(df_index_comment_categories)
# counter

In [23]:
counter

16750

In [24]:
def stack_dataframes(df1, df2):
    if df1.empty:
        return df2.reset_index(drop=True)  # Ensure a clean index reset
    return pd.concat([df1, df2], ignore_index=True)

In [25]:
def tuples_to_json(tuples_list, keys = ["counter", "llm_suggested_categories", "comment"], json_file_path = "C:/Users/shubh/programming_work/git_repos/NU/courses_and_academics/data_mining/proj/comment_classification_data/list_of_tuples_of_skipped_indices_and_comments_3.json"):
    dict_list = [dict(zip(keys, tup)) for tup in tuples_list]
    with open(json_file_path, 'w') as f:
        json.dump(dict_list, f, indent=4)




In [26]:
keys_for_tuples_of_skipped_indices_and_comments = ["counter", "llm_suggested_categories", "comment"]

In [27]:
counter

16750

In [28]:
df_recent_50 = pd.DataFrame()

for comment in comments[counter:40000]:

    prompt = f"{comment}:\n Classify this comment into all the relevant categories from this list: - \n{youtube_comment_categories}\nReturn a list of comma-separated responses."

    # print(prompt)
    
    comment_category_2_bool = dict()
    comment_category_2_bool['unique_id'] = counter
    for cat in youtube_comment_categories:
        comment_category_2_bool[cat] = 0

    message_history = [{"role": "system", "content": "You are a helpful assistant."}]
    message_history.append( { "role": "assistant", "content": prompt } )
    
    llm_suggested_categories = chat_completion_2(message_history, temperature = 0.1).strip("[").strip("]").split(",")
    # llm_suggested_categories = strip_special_chars(llm_suggested_categories)
    # print(llm_suggested_categories)
    # print("\n")
    
    llm_suggested_categories, sanity_check_passed_or_not = check_if_categories_are_proper(llm_suggested_categories, message_history, counter, total_trials = 0)

    if not sanity_check_passed_or_not:
        
        list_of_tuples_of_skipped_indices_and_comments.append( (counter, llm_suggested_categories, comment) )

        tuples_to_json(list_of_tuples_of_skipped_indices_and_comments)

        for category in youtube_comment_categories:
            comment_category_2_bool[category] = 0
    
        list_of_category_2_bools_per_comment.append( comment_category_2_bool )
        counter += 1
        continue
    
    categories_list = [ strip_special_chars(category.strip(" ").strip("'").strip("\n"))  for category in llm_suggested_categories]
    
    for category in categories_list:
        comment_category_2_bool[category] = 1

    list_of_category_2_bools_per_comment.append( comment_category_2_bool )
    counter += 1

    if (not counter%50):
        
        print(f"current length of list_of_category_2_bools_per_comment : {len(list_of_category_2_bools_per_comment)}")
        df_recent_50 = pd.DataFrame( list_of_category_2_bools_per_comment[-50:] )
        
        df_index_comment_categories = stack_dataframes(df_index_comment_categories, df_recent_50)
        
        # df_index_comment_categories = df_index_comment_categories.drop_duplicates()

        df_index_comment_categories.to_csv( file_path, index=False )
        print(f"time out 3 seconds")
        time.sleep(3)


Requires retries - ['Words of empathy', ' Humor and Memes: "Yes', ' I can see how that could be confusing. Let\'s try to clarify it together!"']
Requires retries list - ['Words of empathy', 'Humor and Memes: "Yes', "I can see how that could be confusing. Let's try to clarify it together"]
Requires retries remaining categories - ["I can see how that could be confusing. Let's try to clarify it together", 'Humor and Memes: "Yes']
current length of list_of_category_2_bools_per_comment : 50
time out 3 seconds
Requires retries - ['Personal Stories and Experiences', " Words of empathy\n\nI'm sorry to hear that you realized she didn't pay after 9 years. It must be frustrating to come to that realization after such a long time. If you need any support or advice on how to handle the situation", ' feel free to share more details.']
Requires retries list - ['Personal Stories and Experiences', "Words of empathy\n\nI'm sorry to hear that you realized she didn't pay after 9 years. It must be frustrat

In [29]:
counter

40000